# Coursera_Capstone
### This note book is mainly used for the capstone project

In [1]:
#import packages
import pandas as pd
import pylab as pl
import numpy as np
import seaborn as sns
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [2]:
#read data
df = pd.read_csv("D:\IBM Data Scientist\9  Applied Data Science Capstone\Project dataset\dftRoadSafetyData_Accidents_2018.csv")
df.shape

C:\Users\Shen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(122635, 32)

In [3]:
#drop variables that will not be used
df = df.drop(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 
              'Latitude', 'Date', 'Local_Authority_(District)', 'Local_Authority_(Highway)', 
             '1st_Road_Class', '1st_Road_Number',  '2nd_Road_Class', '2nd_Road_Number', 'Police_Force','Time',
              'LSOA_of_Accident_Location',  'Special_Conditions_at_Site', 
              'Junction_Control','Road_Type'], axis = 1)
df.columns

Index(['Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties',
       'Day_of_Week', 'Speed_limit', 'Junction_Detail',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident'],
      dtype='object')

In [4]:
#delete missing value
df = df[df['Did_Police_Officer_Attend_Scene_of_Accident'] != -1]
df = df[df['Urban_or_Rural_Area'] != -1]
df = df[df['Urban_or_Rural_Area'] != 3]
df = df[df['Junction_Detail'] != -1]
df = df[df['Junction_Detail'] != 9]
df = df[df['Light_Conditions'] != 7]
df = df[df['Weather_Conditions'] != 9]
df = df[df['Weather_Conditions'] != -1]
df = df[df['Weather_Conditions'] != 8]
df = df[df['Road_Surface_Conditions'] != -1]
df = df[df['Carriageway_Hazards'] != -1]
df = df[df['Pedestrian_Crossing-Human_Control'] !=-1]
df = df[df['Pedestrian_Crossing-Physical_Facilities'] !=-1]

#get dummy variable from categorical vairables
df['Junction_Detail'].replace([2,3,4,5,6,7,8], 1, inplace = True)
df['Road_Surface_Conditions'].replace([2,3,4,5], 0, inplace = True)
df['Weather_Conditions'].replace([2,3,4,5,6,7], 0, inplace = True)
df['Light_Conditions'].replace([4,5,6],0, inplace = True)
df['Day_of_Week'].replace([2,3,4,5],1, inplace = True)
df['Day_of_Week'].replace([6,7],0, inplace = True)
df['Carriageway_Hazards'].replace([2,3,4,5,6,7],1, inplace = True)
df['Pedestrian_Crossing-Physical_Facilities'].replace([2,3,4,5,6,7,8],1, inplace = True)
df['Pedestrian_Crossing-Human_Control'].replace(2, 1, inplace = True)
df['Did_Police_Officer_Attend_Scene_of_Accident'].replace([2,3], 0, inplace = True)

In [5]:
df.shape
df['Accident_Severity'].value_counts()

3    84567
2    20920
1     1550
Name: Accident_Severity, dtype: int64

In [6]:
drop_rows_index = df[df['Accident_Severity']==3].index.values
drop_rows_index = drop_rows_index[0:40000]
df = df.drop(drop_rows_index, axis = 0)
df['Accident_Severity'].replace([3,2],[0,1], inplace = True)
df['Accident_Severity'].value_counts()

0    44567
1    22470
Name: Accident_Severity, dtype: int64

In [7]:
#Exploratory data analysis
EDA = df.copy(deep=True)

In [8]:
EDA['Accident_Severity'].replace([1,0],['serious', 'slight'], inplace = True)
EDA['Weather_Conditions'].replace([1,0],['fine', 'not_fine'], inplace = True)
EDA['Light_Conditions'].replace([1,0],['daylight','not_daylight'], inplace=True)
EDA['Road_Surface_Conditions'].replace([1,0],['not_dry','dry'], inplace=True)
EDA['Junction_Detail'].replace([1,0],['at_junctino','not_at_junction'], inplace=True)

In [9]:
EDA1 = EDA.groupby(['Accident_Severity'])['Weather_Conditions'].value_counts(normalize=True)
pd.DataFrame(EDA1)

Weather_Conditions
Accident_Severity Weather_Conditions                    
serious           fine                          0.859769
                  not_fine                      0.140231
slight            fine                          0.845962
                  not_fine                      0.154038

In [10]:
EDA2 = EDA.groupby(['Accident_Severity'])['Light_Conditions'].value_counts(normalize=True)
pd.DataFrame(EDA2)

Light_Conditions
Accident_Severity Light_Conditions                  
serious           daylight                  0.699421
                  not_daylight              0.300579
slight            daylight                  0.763166
                  not_daylight              0.236834

In [11]:
EDA3 = EDA.groupby(['Accident_Severity'])['Road_Surface_Conditions'].value_counts(normalize=True)
pd.DataFrame(EDA3)

Road_Surface_Conditions
Accident_Severity Road_Surface_Conditions                         
serious           not_dry                                 0.748109
                  dry                                     0.251891
slight            not_dry                                 0.729845
                  dry                                     0.270155

In [12]:
EDA4 = EDA.groupby(['Accident_Severity'])['Junction_Detail'].value_counts(normalize=True)
pd.DataFrame(EDA4)

Junction_Detail
Accident_Severity Junction_Detail                 
serious           at_junctino             0.505073
                  not_at_junction         0.494927
slight            at_junctino             0.516705
                  not_at_junction         0.483295

In [13]:
#combine features
numeric_features = df[['Number_of_Vehicles', 'Number_of_Casualties', 'Speed_limit']]
X1 = preprocessing.StandardScaler().fit(numeric_features).transform(numeric_features)
X1 = pd.DataFrame(X1)
X2 = df.drop(['Accident_Severity','Number_of_Vehicles', 'Number_of_Casualties', 'Speed_limit'], axis = 1)
X2 = X2.reset_index(drop=True)

In [14]:
X = pd.concat([X1, X2], axis=1)
y = df['Accident_Severity'].values

In [15]:
#split traning / testing dataset
X_train, X_test, Y_train, Y_test = train_test_split( X, y, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  Y_train.shape)
print ('Test set:', X_test.shape,  Y_test.shape)

Train set: (46925, 13) (46925,)
Test set: (20112, 13) (20112,)


In [16]:
#modeling
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#SVM
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_test, Y_test) * 100, 2)
jaccard_svc = round(jaccard_score(Y_test, Y_pred) * 100, 2)
f1_svc = round(f1_score(Y_test, Y_pred, average='weighted') * 100, 2)

#KNN
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_test, Y_test) * 100, 2)
jaccard_knn = round(jaccard_score(Y_test, Y_pred) * 100, 2)
f1_knn = round(f1_score(Y_test, Y_pred, average='weighted') * 100, 2)

# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
jaccard_logreg = round(jaccard_score(Y_test, Y_pred) * 100, 2)
f1_logreg = round(f1_score(Y_test, Y_pred, average='weighted') * 100, 2)

# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_test, Y_test) * 100, 2)
jaccard_tree = round(jaccard_score(Y_test, Y_pred) * 100, 2)
f1_tree = round(f1_score(Y_test, Y_pred, average='weighted') * 100, 2)

# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_test, Y_test) * 100, 2)
jaccard_rf = round(jaccard_score(Y_test, Y_pred) * 100, 2)
f1_rf = round(f1_score(Y_test, Y_pred, average='weighted') * 100, 2)

In [17]:
#print out the performance for each of the models
print("Machine Learning algorithm scores with weather related conditions")
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Decision Tree'],
    'Accuracy': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_decision_tree],
    'F1 - Score': [f1_svc, f1_knn, f1_logreg, 
              f1_rf, f1_tree],
    'Jaccard - Score': [jaccard_svc, jaccard_knn, jaccard_logreg, 
              jaccard_rf, jaccard_tree]})
models.sort_values(by='Accuracy', ascending=False)

Machine Learning algorithm scores with weather related conditions


,Model,Accuracy,F1 - Score,Jaccard - Score
0,Support Vector Machines,67.80,62.13,16.27
2,Logistic Regression,67.01,57.50,7.23
4,Decision Tree,66.47,61.68,17.01
3,Random Forest,66.34,62.33,18.89
1,KNN,64.45,61.28,19.13
